## NLP: comprehension exercises for word distributions

Matthew Stone
Initial version, Spring 2018  - Revised for Python 3 Spring 2019.

[Zipf's Law][1] offers an appealing way to visualize and quantify the skewed distribution of very frequent events and very rare events in linguistic data (and many other kinds of data).

The setup for this exercise recapitulates the visualization tools that we saw in class for exploring Zipf's Law.

[1]:https://en.wikipedia.org/wiki/Zipf%27s_law

The basic task is just to apply these visualizations to a data set of your choosing.  The structure of this notebook includes
- [Background material](#background) that provides necessary functionality.  (No exercises here, but you'll probably want to read this over to understand the material.)
- [A basic implementation exercise](#basic), where you create a Zipf graph for a data set of your choice.
- [An optional advanced statistical analysis](#advanced), where you use [appropriate statistical tests](http://www.plosone.org/article/info%3Adoi%2F10.1371%2Fjournal.pone.0085777) to appraise the fit of your data to a powerlaw distribution.


### Background Material 
<a id="background"/>

In [1]:
import math
import nltk
import powerlaw
import collections
import numpy
import matplotlib.pyplot as plt
import scipy.stats as ss
import csv
%matplotlib inline
_  = numpy.seterr(divide='ignore', invalid='ignore')

This visualization includes a mode with interactive visualizations with the `plotly` package. They are pretty rewarding, I find, but they are also a little bit slow, because they basically pack a complete corpus into the javascript in the browser. Use them if you like them.

In [1]:
use_plotly = False
if use_plotly:
    import plotly
    import plotly.plotly as py
    import plotly.graph_objs as go
    plotly.offline.init_notebook_mode(connected=True)

If you want to visualize Chinese, Japanese or Korean (CJK) text in matplotlib, you need to load another font.  The following code will probably do the trick for you.

In [ ]:
use_cjk_labels = False
if use_cjk_labels:
    import matplotlib as mpl
    mpl.rc('font', family='SimSun')

We are going to be doing the same computations to visualize the applicability of Zipf's law to a range of data sets, so we create a class Zipf to organize this functionality.

The classic way to visualize Zipf's law is with a scatter plot on a log-log scale. Each item is plotted as a dot at a position  (x,y)(x,y) , where  xx  is its rank (from most common to least common) and  yy  is its actual frequency. Mathematically, you can tell that the data on this kind of scatter plot will all lie on a monotonically decreasing curve: by definition lower ranked entities are more common. However, Zipf's law is reflected in the fact that the curve for linguistic data tends to have a particular shape. In fact, the bulk of the data falls on a line. We'll explain this through examples below. The graph method of a Zipf object displays this graph.

A bunch of other code snippets can be used to help explore a Zipf object z:

- `z.singletons()` a generator function that lists items that occur once in the data set
- `z.n_items_with_rate(k)` count the number of items in the data set that occur k times
- `z.rates.most_common(k)` get a list of the k most common items in the data set
- `z.corr` a matrix giving the correlation coefficient of ranks against log frequencies, typically showing that rank predicts a very large amount of the variance in log frequency.
- `z.describe_singletons()` gives a summary of the frequency of rare events.

To create a `Zipf` object, pass in two arguments: `name` (for labeling graphs), and `rates` (a counter, dictionary or similar structure that associates items with numeric values). The initialization uses the `rankdata` function of scipy to order the items (and reverses the ranks since rankdata goes from lowest to highest), and computes the correlation coefficient. It stores all the results of its computations for later inspection.

In [3]:
class Zipf(object):
    """
    Zipf's law data structure 
    Visualizes the rank-frequency distribution of discrete data
    """
    def __init__(self, name, rates):
        self.name = name
        self.rates = rates
        lowest_to_highest = ss.rankdata([c for (i,c) in rates.items()])
        self.size = len(lowest_to_highest)
        self.ranks = [self.size - r + 1 for r in lowest_to_highest]
        self.rate_data = [c for (i,c) in rates.items()]
        self.labels = [i for (i,c) in rates.items()]
        self.log_data = [math.log(c) for c in self.rate_data]
        self.corr = numpy.corrcoef(self.ranks, self.log_data)
        
    def graph(self):
        plt.figure()
        plt.plot(self.ranks, self.rate_data, 'ro')
        plt.xscale('log')
        plt.xlabel('Frequency rank of item')
        plt.yscale('log')
        plt.ylabel('Number of tokens of item')
        plt.title(u'Rank-frequency distribution for {}'.format(self.name))

    def graphly(self):
        plotly.offline.iplot(dict(data=[go.Scatter(x=self.ranks,
                                                   y=self.rate_data,
                                                   mode='markers',
                                                   text=self.labels, 
                                                   hoverinfo="text")],
                                  layout=dict(title=u'Rank-frequency distribution for {}'.format(self.name),
                                              xaxis=dict(type='log', title='Frequency rank of item'),
                                              yaxis=dict(type='log', title='Number of tokens of item'))))
            

    def show(self) :
        if use_plotly :
            self.graphly()
        else :
            self.graph()
            
    def singletons(self):
        for (i, c) in self.rates.items():
            if c == 1 :
                yield i
    
    def n_items_with_rate(self, rate):
        return sum(1 for (i, c) in self.rates.items() if c == rate)
    
    def describe_singletons(self):
        print(u'In {}, {} of {} tokens are singletons.'.format(self.name, 
                                                              self.n_items_with_rate(1),
                                                              self.size))

## Basic Implementation
<a id="basic"/>

Find a data set that you're curious about that you think very probably has a Zipf distribution.  Visualize it and see if your hunch is correct.

More precisely, in the cell below, you should write code to:
- Load and process the data as necessary to create a counter data structure mapping items in your data set to counts.
- Create a Zipf object based on the counter data structure you have provided.
- Call show() on the resulting object.

Useful sources of data include:
- [Aaron Clauset's archive of power law data sets][1]
- [Project Gutenberg, the premier source of copyright free electronic text][2]
- [US Baby Names][3]   

But by no means feel limited to these!

[1]:http://tuvalu.santafe.edu/~aaronc/powerlaws/data.htm
[2]:https://www.gutenberg.org/
[3]:https://www.ssa.gov/oact/babynames/limits.html

## Advanced Analysis
<a id="advanced"/>

Apply the statistical tests available in the `powerlaw` package to test how well the data set you have visualized actually fits a power law distribution, compared to other heavy tailed distributions.  Use the notebook cells below to carry out and record your analysis (in the code cell) and to summarize and interpret your findings (in the markdown cell).

See [this article for reference.](http://www.plosone.org/article/info%3Adoi%2F10.1371%2Fjournal.pone.0085777)